In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("../")
print(os.getcwd())
from parc.data import data_navier_stokes as data
from parc.model import model_ns as model
!nvidia-smi 

2024-05-24 13:19:36.395304: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 13:19:37.044431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/:/usr/local/cuda/include/:/usr/local/cuda/lib/:/usr/local/cuda/lib64/:/usr/lib/x86_64-linux-gnu/:/opt/apps/TensorRT-8.6.1.6/lib/:/opt/apps/TensorRT-8.6.1.6/include/:/usr/include/x86_64-linux-gnu/:/usr/include/
2024-05-24 13:19:37.044518: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libn

/home/lweger/PARCv2
Fri May 24 13:19:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10                     Off | 00000000:A6:00.0 Off |                    0 |
|  0%   56C    P0              64W / 150W |  22318MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+-----

# Data pipeline

In [2]:
Re_list = [7000,9000]    # all present data
train_list = [9000]    
test_list = [7000]

data_ns = data.DataNS(Re_list)
seq_input = data_ns.import_data(folder_path='./data/ns/')
seq_norm = data_ns.data_normalization(input_data=seq_input, no_of_channel=3)

train_seq, test_seq = data_ns.train_test_split(Re_list=Re_list, Re_test=test_list, Re_train=train_list,input_data=seq_norm[0])
train_data = data_ns.clip_data(input_seq=train_seq,no_of_fields=3,sequence_length=13)

NameError: name 'data' is not defined

# Training

### Stage 1: Differentiator training

In [ ]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,:3])
dataset_label = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,3:])
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 798) 
dataset = dataset.batch(1)


### Stage 2: Integrator training (Can skip)

In [ ]:
tf.keras.backend.clear_session()
parc = model.PARCv2_ns(n_time_step = 12, step_size= 1/38, solver = "heun", mode = "differentiator_training")
# parc.differentiator.load_weights('parc2_diff_ns_heun_10_2.h5')
# parc.poisson.load_weights('parc2_poisson_ns_heun_10_2.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001, beta_1 = 0.5, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

In [ ]:
parc.differentiator.save_weights('parc2_diff_ns_heun_12.h5')

In [ ]:
parc.poisson.save_weights('parc2_poisson_ns_heun_12.h5')

In [ ]:
tf.keras.backend.clear_session()
parc = model.PARCv2_ns(n_time_step = 12, step_size= 1/38, solver = "heun", mode = "integrator_training")
parc.differentiator.load_weights('parc2_diff_ns_heun_12.h5')
parc.poisson.load_weights('parc2_poisson_ns_heun_12.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001, beta_1 = 0.5, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

In [ ]:
parc.integrator.save_weights('parc2_int_ns_heun_12.h5')